# Вызов пользовательских функций в API GigaChat через библиотеку requests
Для начала работы нужно [получить доступ](https://developers.sber.ru/docs/ru/gigachat/api/integration).
## Авторизация
Объявим ключи для авторизации

In [20]:
# Используем вкладку "Секреты" левой панели Colab для хранения ключей
from google.colab import userdata
# auth token API GigaChat
giga_key = userdata.get('GIGA_CHAT_KEY')
# id гугл-таблицы из которой будем получать данные
price_id = userdata.get('PRICE_ID')

Используем для авторизации auth token.

[Документация по авторизации](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/post-token)

In [21]:
import requests
import uuid

def get_token(auth_token, scope='GIGACHAT_API_CORP'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «GIGACHAT_API_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отключать проверку не надо)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1


Получаем токен для запросов

In [22]:
response = get_token(giga_key)
if response != 1:
  print(response.text)
  giga_token = response.json()['access_token']

{"access_token":"eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.KoFDkEc8qUbgUxzA07rjuHGhbEWgJggB0GxPBCo_9TIdIdRtlpuz8qV-NjQvSDFJi6cpeYJe1bxb089Di7M3-vqiqHU8n0MkJrLBfeucRPkZRxlCMcoaWaD8CdFLM5mGYfi2UWZdRbXzTPAoZpDty97y2qkvUXcj3mwHrDBQyDgLf8qk4OFDGhLy1yEa3UxRFW7SOo-wDT4THlEhKhlfuOZu3utQ1TR4HmnLIlKHiw4ZGRBvTsJzP0U9TL3gehWPsfEeQsR1PRwgeVE4E2M1Iii6wSUTrLrB8NjBa7ZTE41Fe2dCt7buonC3qbDYS8KYQ7a0DLjf_mG3JDuMC9ozgQ.ZaRpgisG72y44uaI7UVFLQ.tWicPgLsoQ4Im8TLuVCUX7H6m7pp5v7vpyjh--mlx4SltTS-wHa-0cTTHbFc81noPrlRTuU3W1ixentFzvd6k_vglsuZzHOT6EuLV_Eqc-f9v3w3LLOXewGviXGuyl1dyF3X8CaWxO4OHM8TTQGDcgUk82q6gAhDf-acHIJ9Vsad8R5qfgLYNIuE_zuHZ-7p2C3SIZc-K38BAMhasT3WUaEHhtT7nHxQu3hi-L_zwolC2bQ0YVOyQWDdd4T6foY-w0-MmQ9kjfbESZQeae5b9CEUEC1IMZAcBiuf-8TYRJWtLHE20Xf7Rf3ixtCGXTNkgJ-yWt26T8R8CW3XB7zzy2bKCqp5t-frnyM_g3K5IFy1ToHt1AX1bIjyluMD6IY8hjWBQIy_xHegHC66FFtZjUXODHy12IVUWGqB4tZgZHqeYZveUxAun7S-MDEEY98BqY5J5gGK1Bxj76_P9aumPMP1l4w9R-SN5TkwCrSyUNnmchIVKsoqeajNLWaSc3FasWf2_WdDS4Uqg3KPgWaXCtIrqezkH

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Токен действует 30 минут с момента его получения.

In [4]:
from datetime import datetime

# Временная метка в миллисекундах
timestamp_ms = response.json()["expires_at"]

# Преобразование в секунды
timestamp_s = timestamp_ms / 1000

# Преобразование в объект datetime
date_time = datetime.utcfromtimestamp(timestamp_s)

print(date_time.strftime('%Y-%m-%d %H:%M:%S'))

2024-04-09 10:40:06


## Получение перечня моделей
[Документация](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/get-models)

Вызов функций на данный момент доступен только для модели "GigaChat-Pro-preview", убедитесь, что она доступна на вашем тарифе.

In [5]:
import requests

url = "https://gigachat.devices.sberbank.ru/api/v1/models"

payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': f'Bearer {giga_token}'
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

print(response.text)

{"object":"list","data":[{"id":"GigaChat","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-Plus","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-Pro","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-Pro-preview","object":"model","owned_by":"salutedevices"}]}


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Определим наши пользовательские функции

In [6]:
import pandas as pd

PRICE_LIST_URL = f'https://docs.google.com/spreadsheets/d/{price_id}/export?gid=0&format=csv'
price_list = pd.read_csv(PRICE_LIST_URL)
price_list[['вид продукции',	'цена за 1 кг, рубли РФ']].head()

,вид продукции,"цена за 1 кг, рубли РФ"
0,Тушка индейки самец,295
1,Мясная обрезь грудка индейка,350
2,Мясная обрезь бедро индейка,350
3,Блоки из мяса индейки,400
4,Филе голени индейки без костей,350


In [7]:
def weather_forecast(location):
  """
  Получение погоду в Москве
  """
  return "в Москве +13"

In [8]:
def get_prices(products):
  """
  Получение цен на выбранные продукты
  """
  if isinstance(products, str):
    products = [products]
  return json.dumps(price_list[price_list['вид продукции'].isin(products)][['вид продукции', 'цена за 1 кг, рубли РФ']].to_dict(orient='records'), ensure_ascii=False)

In [9]:
def get_full_pricelist(products):
    """
    Получение информации по всей продукции
    """
    return json.dumps(price_list[['вид продукции', 'цена за 1 кг, рубли РФ']].to_dict(orient='records'), ensure_ascii=False)

In [10]:
functions = [
    {
        "name": "weather_forecast",
        "description": "Возвращает температуру в Москве",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "Местоположение, например, название города"
                    }
            },
            "required": ["location"]
            },
        "return_parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "Местоположение, например, название города"
                    },
                "temperature": {
                    "type": "integer",
                    "description": "Температура для заданного местоположения"
                    },
                "forecast": {
                    "type": "array",
                    "items": {
                        "type": "string"
                        },
                    "description": "Описание погодных условий"
                    },
                "error": {
                    "type": "string",
                    "description": "Возвращается при возникновении ошибки. Содержит описание ошибки"
                    }
                }
            }
        },
        {
            "name": "get_full_pricelist",
            "description": "Возвращает цены на всю имеющуюуся продукцию",
            "parameters": {
                "type": "object",
                "properties": {
                    "products": {
                        "type": "string",
                        "description": f"список наиболее подходящих продуктов из списка {price_list['вид продукции'].to_list()}"
                        }
                    },
                "required": ["products"]
                },
            "return_parameters": {
                "type": "object",
                "properties": {
                    "product": {
                        "type": "string",
                        "description": "Наименование продукции"
                        },
                    "price": {
                        "type": "integer",
                        "description": "Цена продукции в рублях РФ"
                        },
                    "error": {
                        "type": "string",
                        "description": "Возвращается при возникновении ошибки. Содержит описание ошибки"
                        }
                    }
                }
            },
             {
                 "name": "get_prices",
                 "description": "Возвращает цену на запрашиваемый продукт",
                 "parameters": {
                     "type": "object",
                     "properties": {
                         "products": {
                             "type": "string",
                             "description": f"список наиболее подходящих продуктов из списка {price_list['вид продукции'].to_list()}"
                             }
                         },
                     "required": [
                         "products"
                         ]
                     },
                 "few_shot_examples": [
                     {
                         "request": "сколько стоит лопатка?",
                         "params": {
                             "products": ["Свиная лопатка"]
                             }
                         }
                     ],
                 "return_parameters": {
                     "type": "object",
                     "properties": {
                         "products": {
                             "type": "string",
                             "description": "Наименование продукции"
                             },
                         "price": {
                             "type": "integer",
                             "description": "Цена для данного вида продукции"
                             },
                         "error": {
                             "type": "string",
                             "description": "Возвращается при возникновении ошибки. Содержит описание ошибки"
                             }
                         }
                     }
                 }
    ]

In [11]:
import requests
import json

def get_chat_completion(auth_token, user_message, conversation_history=None):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat в рамках диалога.

    Параметры:
    - auth_token (str): Токен для авторизации в API.
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.
    - conversation_history (list): История диалога в виде списка сообщений (опционально).

    Возвращает:
    - response (requests.Response): Ответ от API.
    - conversation_history (list): Обновленная история диалога.
    """
    # URL API, к которому мы обращаемся
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Если история диалога не предоставлена, добавляем в неё системный промт
    if conversation_history is None or conversation_history == []:
        conversation_history = [
            {
                "role": "system",
                "content": "ты менеджер по продажам, твоя задача сообщать цены из прайс-листа и погоду"
                }
            ]

    # Добавляем сообщение пользователя в историю диалога
    conversation_history.append({
        "role": "user",
        "content": user_message
    })
    # json схемы доступных функций
    giga_functions = [
    {
        "name": "weather_forecast",
        "description": "Возвращает температуру в Москве",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "Местоположение, например, название города"
                    }
            },
            "required": ["location"]
            },
        "return_parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "Местоположение, например, название города"
                    },
                "temperature": {
                    "type": "integer",
                    "description": "Температура для заданного местоположения"
                    },
                "forecast": {
                    "type": "array",
                    "items": {
                        "type": "string"
                        },
                    "description": "Описание погодных условий"
                    },
                "error": {
                    "type": "string",
                    "description": "Возвращается при возникновении ошибки. Содержит описание ошибки"
                    }
                }
            }
        },
        {
            "name": "get_full_pricelist",
            "description": "Возвращает цены на всю имеющуюуся продукцию",
            "parameters": {
                "type": "object",
                "properties": {
                    "products": {
                        "type": "string",
                        "description": f"список наиболее подходящих продуктов из списка {price_list['вид продукции'].to_list()}"
                        }
                    },
                "required": ["products"]
                },
            "return_parameters": {
                "type": "object",
                "properties": {
                    "product": {
                        "type": "string",
                        "description": "Наименование продукции"
                        },
                    "price": {
                        "type": "integer",
                        "description": "Цена продукции в рублях РФ"
                        },
                    "error": {
                        "type": "string",
                        "description": "Возвращается при возникновении ошибки. Содержит описание ошибки"
                        }
                    }
                }
            },
             {
                 "name": "get_prices",
                 "description": "Возвращает цену на запрашиваемый продукт",
                 "parameters": {
                     "type": "object",
                     "properties": {
                         "products": {
                             "type": "string",
                             "description": f"список наиболее подходящих продуктов из списка {price_list['вид продукции'].to_list()}"
                             }
                         },
                     "required": [
                         "products"
                         ]
                     },
                 "few_shot_examples": [
                     {
                         "request": "сколько стоит лопатка?",
                         "params": {
                             "products": ["Свиная лопатка"]
                             }
                         }
                     ],
                 "return_parameters": {
                     "type": "object",
                     "properties": {
                         "products": {
                             "type": "string",
                             "description": "Наименование продукции"
                             },
                         "price": {
                             "type": "integer",
                             "description": "Цена для данного вида продукции"
                             },
                         "error": {
                             "type": "string",
                             "description": "Возвращается при возникновении ошибки. Содержит описание ошибки"
                             }
                         }
                     }
                 }
    ]
    # Подготовка данных запроса в формате JSON
    payload = json.dumps({
        "model": "GigaChat-Pro-preview",
        "messages": conversation_history,
        "function_call": "auto",
        "functions": giga_functions,
        "temperature": 0.5,
        "top_p": 0.1,
        "n": 1,
        "stream": False,
        "max_tokens": 32000,
        "repetition_penalty": 1,
        "update_interval": 0
    })
    # Заголовки запроса
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {auth_token}'
    }
    # словарь с функциями для обработки запроса
    available_functions = {
        "weather_forecast": weather_forecast,
        "get_full_pricelist": get_full_pricelist,
        "get_prices": get_prices
        }
    # Выполнение POST-запроса и возвращение ответа
    try:
        response = requests.post(url, headers=headers, data=payload, verify=False)
        response_data = response.json()
        # проверяем ответ модели на наличие обращений к функциям
        try:
          # json с информацией о необходимой функции
          func_calls = response_data['choices'][0]['message']['function_call']
          # имя вызываемой функции
          func_name = func_calls['name']
          # аргументы вызываемой функции
          func_args = func_calls['arguments']
          # достаём нужную функцию из словаря
          function_to_call = available_functions[func_name]

          # добавляем в историю сообщений ответ модели с вызовом функции, БЕЗ ЭТОГО МОДЕЛЬ НЕ ОТВЕТИТ
          conversation_history.append(response_data['choices'][0]['message'])
          # добавляем в историю сообщений результаты функции
          conversation_history.append(
              {
                  "role": "function",
                  "content": function_to_call(**func_args),
                  "name": func_name
                  }
              )
          # обновляем данные
          payload = json.dumps({
              "model": "GigaChat-Pro-preview",
              "messages": conversation_history,
              "function_call": "auto",
              "functions": giga_functions,
              "temperature": 0.5,
              "top_p": 0.1,
              "n": 1,
              "stream": False,
              "max_tokens": 32000,
              "repetition_penalty": 0.5,
              "update_interval": 0
              })
          # повторяем зарос
          response = requests.post(url, headers=headers, data=payload, verify=False)
          response_data = response.json()
          # for func in func_calls:
        except:
          pass

        # Добавляем ответ модели в историю диалога
        conversation_history.append({
            "role": "assistant",
            "content": response_data['choices'][0]['message']['content']
        })

        return response, conversation_history
    except requests.RequestException as e:
        # Обработка исключения в случае ошибки запроса
        print(f"Произошла ошибка: {str(e)}")
        return None, conversation_history

In [12]:
conversation_history = []

response, conversation_history = get_chat_completion(giga_token, "какая погода в Москве?", conversation_history)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [13]:
conversation_history

[{'role': 'system',
  'content': 'ты менеджер по продажам, твоя задача сообщать цены из прайс-листа и погоду'},
 {'role': 'user', 'content': 'какая погода в Москве?'},
 {'content': '',
  'role': 'assistant',
  'function_call': {'name': 'weather_forecast',
   'arguments': {'location': 'Москва'}}},
 {'role': 'function', 'content': 'в Москве +13', 'name': 'weather_forecast'},
 {'role': 'assistant', 'content': 'В Москве сейчас +13°С.'}]

In [14]:
conversation_history = []

response, conversation_history = get_chat_completion(giga_token, "дай цены по всей продукции", conversation_history)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [15]:
conversation_history

[{'role': 'system',
  'content': 'ты менеджер по продажам, твоя задача сообщать цены из прайс-листа и погоду'},
 {'role': 'user', 'content': 'дай цены по всей продукции'},
 {'content': '',
  'role': 'assistant',
  'function_call': {'name': 'get_full_pricelist',
   'arguments': {'products': 'всю'}}},
 {'role': 'function',
  'content': '[{"вид продукции": "Тушка индейки самец", "цена за 1 кг, рубли РФ": "295"}, {"вид продукции": "Мясная обрезь грудка индейка", "цена за 1 кг, рубли РФ": "350"}, {"вид продукции": "Мясная обрезь бедро индейка", "цена за 1 кг, рубли РФ": "350"}, {"вид продукции": "Блоки из мяса индейки", "цена за 1 кг, рубли РФ": "400"}, {"вид продукции": "Филе голени индейки без костей", "цена за 1 кг, рубли РФ": "350"}, {"вид продукции": "Медальон (Деликатесный) индейка", "цена за 1 кг, рубли РФ": "350"}, {"вид продукции": "Голень индейки самец", "цена за 1 кг, рубли РФ": "190"}, {"вид продукции": "Голень индейки самка", "цена за 1 кг, рубли РФ": "160"}, {"вид продукции": 

In [ ]:
conversation_history = []

response, conversation_history = get_chat_completion(giga_token, "сколько стоит лопатка и окорок?", conversation_history)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
conversation_history

In [17]:
conversation_history = []

response, conversation_history = get_chat_completion(giga_token, "сколько стоит рагу и какая погода в москве?", conversation_history)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [18]:
conversation_history

[{'role': 'system',
  'content': 'ты менеджер по продажам, твоя задача сообщать цены из прайс-листа и погоду'},
 {'role': 'user', 'content': 'сколько стоит рагу и какая погода в москве?'},
 {'content': '',
  'role': 'assistant',
  'function_call': {'name': 'get_prices',
   'arguments': {'products': 'Рагу индейки  (монолит), Рагу индейки (лоток)'}}},
 {'role': 'function', 'content': '[]', 'name': 'get_prices'},
 {'role': 'assistant',
  'content': 'Извините, но я не могу предоставить вам информацию о цене рагу. Могу ли я помочь вам с чем-то ещё?'}]